In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import email
import email.policy
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

os.listdir('/Users/mounicakolukula/Downloads/Project/archive/hamnspam/')

hamFiles = [name for name in sorted(os.listdir('/Users/mounicakolukula/Downloads/Project/archive/hamnspam/ham/'))]
spamFiles = [name for name in sorted(os.listdir('/Users/mounicakolukula/Downloads/Project/archive/hamnspam/spam/'))]

len(hamFiles)

2551

In [3]:
len(spamFiles)

501

In [4]:
def viewEmail(isSpam, filename):
    folder = "/Users/mounicakolukula/Downloads/Project/archive/hamnspam/spam/" if isSpam else "/Users/mounicakolukula/Downloads/Project/archive/hamnspam/ham/"
    with open(os.path.join(folder, filename), "rb") as fl:
        return email.parser.BytesParser(policy=email.policy.default).parse(fl)
    
hamEmails = [viewEmail(isSpam=False, filename=name) for name in hamFiles]
spamEmails = [viewEmail(isSpam=True, filename=name) for name in spamFiles]

firstHamEmail = hamEmails[0]
firstHamEmail.get_content_type()
firstHamEmail.is_multipart()
firstHamEmail.get_payload()
firstHamEmail.keys()
firstHamEmail.values()
print(firstHamEmail.get_content())


    Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55

In [5]:
print(spamEmails[75].get_content())

*****Write down*****


Hello ,


It is time to refinance!

Your credit does not matter, we can approve anyone.

Now is the time to let some of the top mortgage companies 
in the country compete for your business. 

If you have good credit we will give you the most amazing 
rates available anywhere! 

If you have poor credit, don't worry! We can still refinance 
you with the most competitive rates in the industry!


Let Us put Our Expertise to Work for You! Guaranteed!

http://21377@www.top-lenders.com/app



Best,
Top-Lenders









Erase
http://21631@www.top-lenders.com/remove.html




In [6]:
my_cols = ['To','From','Subject','Content-Type']

df = pd.DataFrame()
for i in range(len(hamEmails)):
    try:
        email_df =pd.DataFrame(hamEmails[i].values(),hamEmails[i].get_body()).T
        email_df = email_df[my_cols]
        email_df['body'] =hamEmails[i].get_content()
        df= df.append(email_df)
    except:
        pass


df.reset_index(inplace= True)
df.drop('index',axis=1,inplace = True)
df['isSpam'] = 0
df5 = pd.DataFrame({'To': ["ritika.johri@gmail.com",
                           "ritika.johri@gmail.com",
                           "ritika.johri@gmail.com",
                           "ritika.johri@gmail.com",
                           "ritika.johri@gmail.com",
                           "mounica168@gmail.com",
                           "mounica168@gmail.com",
                           "mounica168@gmail.com",
                           "mounica168@gmail.com",
                           "ritika.johri@gmail.com",
                           "mounica168@gmail.com",
                           "ritika.johri@gmail.com",
                           "shishira.v.kasal@gmail.com",
                           "manishdm123@gmail.com",
                           "shishira.v.kasal@gmail.com",
                           "shishira.v.kasal@gmail.com",
                           "ritika.johri@gmail.com"],
                    'From':["AppleStore <internationalplaza@apple.com>",
                            "Wells Fargo <alerts@notify.wellsfargo.com>",
                            "thelodgeatlakecrest@emailrelay.com",
                            "Apple <noreply@email.apple.com>",
                            "ISKCON Tampa <iskcon.tampabay@gmail.com>",
                            "tatacom-jobnotification@noreply.jobs2web.com",
                            "Alumni Association of Gudlavalleru Engineering College <notifications@myalumni.network>",
                            "HackerRank Team <no-reply@hackerrankmail.com>",
                            "eBay <ebay@ebay.com>",
                            "Indeed <no-reply@indeed.com>",
                            "Swiggy <alerts@info.swiggy.in>",
                            "Amazon.in <order-update@amazon.in>",
                            "events@deeplearning.ai",
                            "TreasureIsland@public.govdelivery.com",
                            "Amazon.com <auto-confirm@amazon.com>",
                            "Google <no-reply@accounts.google.com>",
                            "Amazon.com <auto-confirm@amazon.com>"],
                    'Subject':["Your receipt from Apple International Plaza",
                               "Wells Fargo online statement ready to view",
                               "Reminder: Rent is Due Today",
                               "Your Apple ID was used to sign in to iCloud on an Apple Watch SE.",
                               "KARTHIK MONTH SPIRITUAL PICNIC - SUNDAY, November 06, 2022 - 11 AM Onwards - Registration Required",
                               "New jobs posted from jobs.tatacommunications.com",
                               "Monthly Digest - 1st Nov 2022",
                               "Practice Coding with Japanese Cities' Names",
                               "dell inspiron 15 display screen 3567: 2 NEW!",
                               "Professional Resume Help by Indeed",
                               "The Leaning Power of Pizza",
                               "Delivered: Your Amazon package has been delivered.",
                               "[Upcoming Event] Branching out of the Notebook: ML Application Development with GitHub",
                               "Halloween Events Happening in Treasure Island",
                               "Your Amazon.com order #114-44305",
                               "Security alert for manishdm123@gmail.com",
                               "Your Amazon.com order #113-2744877-0501834"],
                    'Content-Type':['text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"',
                                   'text/plain; charset="us-ascii"'],
                    'body':["Thank you for shopping at the Apple Store.To tell us about your experience, click here.",
                           "Your new statement is now available online The new statement for your Wells Fargo deposit account XXXXXX7404 is now available to view online. To view your statement: Sign on to Statements and Documents.Select Statements and Disclosures.Choose your account from the dropdown menu.If you have questions about your account, please refer to the contact information on your statement. For questions about viewing your statements online, Wells Fargo Customer Service is available 24 hours a day, 7 days a week. Call us at 1-800-956-4442.",
                           "Hello Neighbors,Here's a friendly reminder that rent is due today. Please review the following information to make sure your account is in good standing. If you have already paid rent for this month, feel free to disregard this message.1st and 2nd Day of the Month Rent is always due by the 1st and 2nd day of the month, every month – this never changes! *If you have reoccurring payments set up: double check that your payment was processed! 3rd Day of the Month ++ Any payments time-stamped on the 3rd or after receive a $125 late fee. Starting on the 3rd of the month, you cannot make partial payments. Any partial payments will have their accounts blocked and will only be able to pay rent by money order/with certified funds in full. We can’t accept personal checks after this date.",
                           "Dear Ritika Johri,Your Apple ID (ritika.johri@gmail.com) was used to sign in to iCloud on an Apple Watch SE. Date and Time: November 2, 2022, 2:37 AM UTC If the information above looks familiar, you can ignore this message. If you have not recently signed in to an Apple Watch SE with your Apple ID and believe someone may have accessed your account, go to Apple ID and change your password as soon as possible. Apple Support",
                           "Dear Friend, ISKCON Tampa humbly invites you and your dear ones to attend the Karthik Month Special Spiritual Picnic at Hillsborough River State Park this Saturday. We will be fortunate to have the wonderful association of senior vaishnavas during this event. When -  Sunday, November 06, 2022 Time -  11 AM - 3 PM EST Location - 15402 US-301N, Thonotosassa, FL 33592 (Hillsborough State Park) Shelter Number - 9 This is a FREE event but  *** REGISTRATION IS MANDATORY *** Please complete your registrations using the link below by November 03, 5 PM. You will get an opportunity to OFFER LAMP to Lord Damodar, participate in Kirtan, hear Krishna Katha, take the association of senior devotees and enjoy sumptuous Lunch prasadam all in the midst of mother nature. The kids will be engaged in spiritual playtime as well. Please join us for a day filled with spiritual bliss.At Your Service ISKCON TAMPA",
                           "You are receiving this email because you joined the Tata Communications Talent Community on 15/10/20. You will receive these messages every 7 day(s). Your Job Alert matched the following jobs at jobs.tatacommunications.com. Jobs Jr. Team Member - Service Engineering - 11001 - Chennai, India",
                           "Alumni Association of Gudlavalleru Engineering College aagec.gecgudlavalleru.ac.in",
                           "Hi Mounica, Improve your skills with this challenge recommended for you: Japanese Cities' Names SQL | 1,143,980 submissions In this challenge, you will query a list of all the Japanese cities' names. Solve Challenge",
                           "Dell Inspiron Display Screen 3567: 2 new matches today. Similar items you may be interested in. Load more.",
                           "We offer help from professionals to take the guesswork out of resume editing. You can request professional feedback on your resume and we'll send you a custom 10 minute video review that will help you. If you want, we can also write a new resume for you! Get Started!",
                           "This offer is so cheesy, we bet you will lean in.",
                           "Hi Ritika,Your package has been delivered!Please rate your delivery experience. Return or replace items in Your Orders.  Order # 171-7779209-8210756. This email was sent from an email address that can't receive emails. Please don't reply to this email. Hi Ritika, Your package has been delivered! Please rate your delivery experience.",
                           "Dear Friends, Join us for a live, interactive workshop that teaches you how to use code from an existing containerized ML web application to deploy a new open-source text-to-image diffusion model for internal stakeholders. This hands-on session will cover how to accomplish this task using version control best practices, including GitHub branch development and pull requests. MacOS will be used for the demo, and all setup and deployment codes will be provided. Those with a non-technical background are welcome to attend. Special thanks to FourthBrain for co-hosting the event with us! Can't attend live? Don't worry. For anyone registered, we will be sending out the recorded sessions afterward.",
                           "We're ready to have a 'fab-BOO-lous' Halloween weekend in Treasure Island, and can't wait for you all to join us! Trick-or-Treat Trail | Friday, Oct. 28: Grab a pail and come out to the Trick-or-Treat trail from 6-8 p.m. at the Treasure Bay Golf and Tennis facility, 10315 Paradise Boulevard. Activities include trick-or-treating, inflatables, DJ, interactive entertainers, and a costume contest at 6:30 p.m.",
                           "Thank you for shopping with us. We’ll send a confirmation when your item ships.",
                           "Google A new sign-in on Mac shishira.v.kasal@gmail.com. We noticed a new sign-in to your Google Account on a Mac device. If this was you, you don’t need to do anything. If not, we’ll help you secure your account. Check activity",
                           "Order Confirmation Hello Ritika, Thank you for shopping with us. We’ll send a confirmation when your item ships. Details Order #113-2744877-0501834 Arriving:Monday, November 7 View or manage order Ship to: Ritika TAMPA, FL Order Total: $20.41 This email was sent from a notification-only address that cannot accept incoming email. Please do not reply to this message."],
                    'isSpam':[0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0,
                             0]
})

df = pd.concat([df, df5])
df.reset_index(inplace= True)

#df.drop('level_0',axis=1,inplace = True)
df.drop('index',axis=1,inplace = True)
df.tail(10)


,To,From,Subject,Content-Type,body,isSpam
2041,mounica168@gmail.com,HackerRank Team <no-reply@hackerrankmail.com>,Practice Coding with Japanese Cities' Names,"text/plain; charset=""us-ascii""","Hi Mounica, Improve your skills with this chal...",0
2042,mounica168@gmail.com,eBay <ebay@ebay.com>,dell inspiron 15 display screen 3567: 2 NEW!,"text/plain; charset=""us-ascii""",Dell Inspiron Display Screen 3567: 2 new match...,0
2043,ritika.johri@gmail.com,Indeed <no-reply@indeed.com>,Professional Resume Help by Indeed,"text/plain; charset=""us-ascii""",We offer help from professionals to take the g...,0
2044,mounica168@gmail.com,Swiggy <alerts@info.swiggy.in>,The Leaning Power of Pizza,"text/plain; charset=""us-ascii""","This offer is so cheesy, we bet you will lean in.",0
2045,ritika.johri@gmail.com,Amazon.in <order-update@amazon.in>,Delivered: Your Amazon package has been delive...,"text/plain; charset=""us-ascii""","Hi Ritika,Your package has been delivered!Plea...",0
2046,shishira.v.kasal@gmail.com,events@deeplearning.ai,[Upcoming Event] Branching out of the Notebook...,"text/plain; charset=""us-ascii""","Dear Friends, Join us for a live, interactive ...",0
2047,manishdm123@gmail.com,TreasureIsland@public.govdelivery.com,Halloween Events Happening in Treasure Island,"text/plain; charset=""us-ascii""",We're ready to have a 'fab-BOO-lous' Halloween...,0
2048,shishira.v.kasal@gmail.com,Amazon.com <auto-confirm@amazon.com>,Your Amazon.com order #114-44305,"text/plain; charset=""us-ascii""",Thank you for shopping with us. We’ll send a c...,0
2049,shishira.v.kasal@gmail.com,Google <no-reply@accounts.google.com>,Security alert for manishdm123@gmail.com,"text/plain; charset=""us-ascii""",Google A new sign-in on Mac shishira.v.kasal@g...,0
2050,ritika.johri@gmail.com,Amazon.com <auto-confirm@amazon.com>,Your Amazon.com order #113-2744877-0501834,"text/plain; charset=""us-ascii""","Order Confirmation Hello Ritika, Thank you for...",0


In [7]:
df6_spam_personal_mails = pd.DataFrame({
                       "To": ["mounica168@gmail.com",  
                              "mounica168@gmail.com",  
                              "Mounica Kolukula <mounica168@gmail.com>", 
                              "Mounica Kolukula <mounica168@gmail.com>", 
                              "mounica168@gmail.com", 
                              "ritika.johri@gmail.com"" <ritika.johri@gmail.com>", 
                              "ritika.johri@gmail.com"" <ritika.johri@gmail.com>", 
                              "ritika.johri@gmail.com"" <ritika.johri@gmail.com>", 
                              "Ritika Aditya Johri <ritika.johri@gmail.com>", 
                              "ritika.johri@gmail.com", 
                              "ritika.johri@gmail.com", 
                              "mounica168@gmail.com",  
                              "Mounica Kolukula <mounica168@gmail.com>", 
                              "mounica168@gmail.com",  
                              "shishira.v.kasal@gmail.com", 
                              "manishdm123@gmail.com", 
                              "manishdm123@gmail.com", 
                              "shishira.v.kasal@gmail.com", 
                              "manishdm123@gmail.com", 
                              "shishira.v.kasal@gmail.com", 
                              "ritika.johri@gmail.com"], 
                   "From": [ 
                       "redBus <greetings@travel.e-redbus.in>", 
                       "Intelius <accounts@mg.mailer.intelius.com>", 
                       "info@infoamcat.com", 
                       "Jakki Jakaj <jakki.jakaj@dbtlabs.com>", 
                       "Team KodNest <hello@kodnest.com>",
                       "Smart TV <info@am.dealsboat.com>", 
                       "Visa Interview <info@am.foxletters.com>", 
                       "OnePlus Sale <info@am.foxletters.com>", 
                       "Queens College - Office of Graduate Admissions <grad@qc.cuny.edu>", 
                       "Thailand Jobs <reply@indiamailers.com>", 
                       "Card Payments <reply@indiamailers.com>", 
                       "training_campaigns@internshala.com", 
                       "Divya Singh <jubin@gradjago.com>", 
                       "Harish Pandya <jubin@gradjago.com>", 
                       "Zipongo <hello@zipongo.com>", 
                       "Lume Deodorant <lmd45567@uve6mhshj136qu.wd8f-ef2f.ledeyu.us>", 
                       "ICICI Pru <info@mailmx1.reachmantra.com>", 
                       "HelloFreshAdPartner <mhlf45535@1ye6ltrtg617if.w15e8-30bc.hukuse.com>", 
                       "Support <manishdm123@188329122679564179.com>", 
                       "Maus Nissan of North Tampa <sender@drivecampaigns.com>", 
                       "Stock Dividend <reply@indiamailers.com>" 
                   ], 
                   "Subject": [ 
                       "Mounica, Your outstation rYde just got lighter on the pocket!", 
                       "Who was (626) 780-3949? Find out now!", 
                       "No hustles, no waiting, only great opportunities – says this AMCATeer", 
                       "Last chance: Monday is the big day", 
                       "What KodNest is offering?", 
                       "Redmi Smart TV X86 Announced in India", 
                       "US to start Visa Interviews from THIS Date", 
                       "Diwali Head Start Sale with Great Discounts", 
                       "Explore Queens College", 
                       "Govt warns against Fake Jobs in Thailand", 
                       "Bank Guidelines for Missed Credit Card Payment", 
                       "Mounica, high competition? Get an edge with this", 
                       "You are selected for Developer hiring", 
                       "Assured placements with IBM cert program", 
                       "Share Your Feedback – Win an Instant Pot", 
                       "Aluminum-free 72 hr odor control?", 
                       "Give your child a secured future.", 
                       "Find your perfect weekly HelloFresh custom meal plan for 65% off your first box", 
                       "Payment Report - 923565-6510-200", 
                       "No Tricks All Treats", 
                       "10 Stocks that Payed Biggest Dividend in 2022" 
                   ], 
                   "Content-Type": ['text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"', 
                                    'text/plain; charset="us-ascii"' 
                                   ], 
                   "body": ["Enjoy sweet savings and spend a bit more on that midway stop-over on your outstation trip! Book now",
                            "Hi Mounica, Come back to Intelius® and see what we've found! Our comprehensive report on (626) 780-3949 is complete and waiting for you. Click the button below to complete your search and see what we’ve uncovered! Your results could reveal",
                            "Shivani decided to opt for AMCAT because she had heard a lot about it; how good companies regularly hire through AMCAT and the huge number of opportunities a good score can bring. As expected, Shivani was soon shortlisted for a lot of companies offering salary packages ranging from 4 LPA to 12 LPA. Read the full success story of Shivani,",
                            "Hey Mounica, There is still time to join us IRL in New Orleans, Sydney, and London. But, If you can't make it in person, take advantage of the online experience you know and love from the comfort of your home. Save my seat!",
                            "Dear Student, Greetings from KodNest. As a fresher it is always important to find the right career path for your First Dream Job. The usual confusion that you would have at an early stage is whether you need to take up development or testing paths? It is very simple. If you love coding then choose development or if you love Testing choose testing. Do not choose something because someone else is choosing it, choose because you like them. We will take care of you becoming a master and all that matters for us is your dedication and commitment to learning.",
                            "Smart TV Redmi Smart TV X86 Announced in India Xiaomi’s Redmi has introduced the Smart TV X86. It has an 86-inch screen, a housing that measures 192 x 110 metres, MEMC support and more. The native chip decodes 4K HDR with clean picture layering and vivid light.Know MoreSee Full Specs - see here- All Rights Reserved -",
                            "Visa Interview :US to start Visa Interviews from THIS DateA senior American diplomat has told Indian community leaders that United States will restart in-person interview requirements for many visa applicants, including stude......expandKnow MoreInterview Dates - see here- All Rights Reserved",
                            "OnePlus Sale :Diwali Head Start Sale with Great Discounts Amazon as well as OnePlus are ready to sell OnePlus Phones with amazing discount in India. It’s the most powerful smartphone brand available.......expand Know More Spec Details - see here- All Rights Reserved -",
                            "Dear Ritika Aditya, Enhance your career prospects with a graduate degree from Queens College. Queens College offers more than 100 master’s degrees and advanced certificate programs that are tailored to the demands of the job market, ranging across the liberal arts, sciences, social sciences, business and much more. You will learn from award-winning professors and benefit from our flexible class schedules and affordable tuition. A QC degree opens doors to a world of possibilities. Our graduates excel in careers with leading companies and organizations, including Bloomberg, J.P. Morgan, Con Edison, Sony Pictures, NYU Langone, Northwell Health, Universal Music Group, NYC Department of Education, NYC Commission for the United Nations, and PwC. New this year are the Queens College School of Business and the School of Arts, which offer the best of the liberal arts while building on emerging trends and our vast network of business partners and community organizations. Our tree-lined 80-acre campus features a traditional quad and a state-of-the-art library and study labs – which makes for an exceptional learning environment. Want to live where you learn? Our beautiful residence hall, the Summit Apartments, provides fantastic amenities for an on-campus experience. Ready to apply? Get started here! Thank you for considering Queens College as you take the next step towards your future. Sincerely,Suzana Pavisic Director of Graduate Admissions Queens College of the City University of New York",
                            "Thailand Jobs : Govt warns against Fake Jobs in Thailand As per a recent report, around 32 nationals of India were rescued by the country’s authorities, who were left stranded in a remote part of Myanm.........expand Know More Official Alert - see here © All Rights Reserved",
                            "Card Payments : Bank Guidelines for Missed Credit Card Payment There are times when you intend to pay the credit payment on time but fail to do so because of an emergency or either because you forgot abo.......expand Know More Avoid Late Fee - see here © All Rights Reserved",
                            "Internshala Trainings Give flight to your ambitions Internshala (Scholiverse Educare Pvt. Ltd.) 610, Unitech Business Zone Golf Course Extension Road, Sector 50, Gurugram Want learning opportunities delivered to your inbox? Click here",
                            "Guaranteed job program in full stack development. Highest salary offered.",
                            "Hello Kolukula Mounica, Data is good team hopes you are doing well Once you have successfully completed our hiring process, you will receive a job guarantee. My expertise in  Data Science and Artificial Intelligence qualifies me to provide this service. Enrol immediately as IBM Corporation has partnered with this program to help you increase the value of your profile in the industry. In this way, recruiters will recognize you as a top candidate. Register Today",
                            "We want to hear from you! Take this short survey to be entered into our monthly drawing to win an Instant Pot ENTER NOW It takes less than 2 minutes, and you could win the seriously impressive cooking tool that’s a pressure cooker, slow cooker, rice cooker, sauté pan, yogurt maker and warming pot all in one. Thanks! The Foodsmart by Zipongo Team",
                            "Special Offer Lumē Starter Pack GET OFFER Clinically Proven, Outrageously Effective Whole Body Deodorant For Pits, Feet & Privates",
                            "CLICK HERE to subscribe",
                            "Take the stress out of mealtime 65% Off + Free Shipping On Your First Box Plus 20% Off of 2nd - 4th Boxes Unlock Offer",
                            "You transfer 24 Bitcoin to me and once the transfer is received, I will delete all this dirty stuff right away. After that, we will forget about each other. I also promise to deactivate and delete all the harmful software from your devices. Trust me. I keep my word. That is a fair deal, and the price is relatively low, considering that I have been checking out your profile and traffic for some time by now. If you don't know how to purchase and transfer Bitcoin - you can use any modern search engine. You need to send that amount here Bitcoin wallet: (The price is not negotiable). You have 3 days in order to make the payment from the moment you opened this email.",
                            "No Payments Until 2023! Sell Us Your Car For Cash!",
                            "Stock Dividend : 10 Stocks that Payed Biggest Dividend in 2022 ritika.johri@gmail.com The fundamentally strongest companies have rock solid cash flows. They often share this cash with investors as dividends. The best compani.........expand Know More For Company List - see here"],
                   "isSpam":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1] 
                                       }) 

dfSpam = pd.DataFrame()
for i in range(len(spamEmails)):
    try:
        s_email_df =pd.DataFrame(spamEmails[i].values(),spamEmails[i].get_body()).T
        s_email_df = s_email_df[my_cols]
        s_email_df['body'] =spamEmails[i].get_content()
        dfSpam= dfSpam.append(s_email_df)
    except:
        pass

dfSpam.reset_index(inplace= True)
dfSpam.drop('index',axis=1,inplace = True)
dfSpam['isSpam'] = 1

dfSpam

,To,From,Subject,Content-Type,body,isSpam
0,dcek1a1@netsgo.com,12a1mailbot1@web.de,Life Insurance - Why Pay More?,"text/html; charset=""iso-8859-1""","<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",1
1,ilug@linux.ie,Slim Down <taylor@s3.serveimage.com>,[ILUG] Guaranteed to lose 10-12 lbs in 30 days...,"text/plain; charset=""iso-8859-1""",1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,zzzz@example.com,Slim Down <sabrina@mx3.1premio.com>,Guaranteed to lose 10-12 lbs in 30 days ...,"text/plain; charset=""iso-8859-1""",1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,zzzz@example.com,Account Services <wsup@playful.com>,Re: Fw: User Name & Password to Membership To ...,"text/plain; charset=""iso-8859-1""",##############################################...,1
4,social@linux.ie,Slim n Trim <yenene@mx2.1premio.com>,[ILUG-Social] re: Guaranteed to lose 10-12 lbs...,"text/plain; charset=""iso-8859-1""",I thought you might like these:\n1) Slim Down ...,1
...,...,...,...,...,...,...
353,"ejennifer3@uswest.net, as99@hotmail.com, nvg_c...",2002biz2biz2513@Flashmail.com,FWD:Direct marketing is working ...,"text/plain; charset=""Windows-1252""",There is NO stumbling on to it! \n\nThe greate...,1
354,"ejennifer3@uswest.net, as99@hotmail.com, nvg_c...",biz2biz2446@Flashmail.com,See your Company sales sky rocket. ...,"text/plain; charset=""Windows-1252""",There is NO stumbling on to it! \n\nThe greate...,1
355,undisclosed-recipients:;,Answer.Us@davicom.co.kr,$10 a hour for watching e-mmercials! No joke!,text/html,"<html>\n\n<head>\n<meta http-equiv=""Content-La...",1
356,Mike@dogma.slashnull.org,beautyinfufuxxxmeb13mxy@aol.com,Make a Fortune On eBay ...,"text/html; charset=""iso-8859-1""","<HTML>\n<BODY>\n <tr valign=""top""> \n <td ...",1


In [8]:
df=df[df['Content-Type']=='text/plain; charset="us-ascii"']
all_data=df.append(dfSpam)
all_data

,To,From,Subject,Content-Type,body,isSpam
0,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,Robert Elz <kre@munnari.OZ.AU>,Re: New Sequences Window,"text/plain; charset=""us-ascii""","Date: Wed, 21 Aug 2002 10:54:46 -05...",0
3,undisclosed-recipient:;,Monty Solomon <monty@roscom.com>,[IRR] Klez: The Virus That Won't Die,"text/plain; charset=""us-ascii""",Klez: The Virus That Won't Die\n \nAlready the...,0
11,Chris Garrigues <cwg-dated-1030376441.95dd45@D...,Robert Elz <kre@munnari.OZ.AU>,Re: New Sequences Window,"text/plain; charset=""us-ascii""","Date: Wed, 21 Aug 2002 10:40:39 -05...",0
14,linux-raid@vger.kernel.org,"""John P. Looney"" <valen@tuatha.org>",[ILUG] Re: Problems with RAID1 on cobalt raq3,"text/plain; charset=""us-ascii""","On Mon, Aug 19, 2002 at 03:08:16PM +0100, John...",0
18,"Kiall Mac Innes <kiall@redpie.com>, ILUG <ilug...",Fergal Moran <fergal.moran@wasptech.com>,RE: [ILUG] Sun Solaris..,"text/plain; charset=""us-ascii""",In a nutshell - Solaris is Suns own flavour of...,0
...,...,...,...,...,...,...
353,"ejennifer3@uswest.net, as99@hotmail.com, nvg_c...",2002biz2biz2513@Flashmail.com,FWD:Direct marketing is working ...,"text/plain; charset=""Windows-1252""",There is NO stumbling on to it! \n\nThe greate...,1
354,"ejennifer3@uswest.net, as99@hotmail.com, nvg_c...",biz2biz2446@Flashmail.com,See your Company sales sky rocket. ...,"text/plain; charset=""Windows-1252""",There is NO stumbling on to it! \n\nThe greate...,1
355,undisclosed-recipients:;,Answer.Us@davicom.co.kr,$10 a hour for watching e-mmercials! No joke!,text/html,"<html>\n\n<head>\n<meta http-equiv=""Content-La...",1
356,Mike@dogma.slashnull.org,beautyinfufuxxxmeb13mxy@aol.com,Make a Fortune On eBay ...,"text/html; charset=""iso-8859-1""","<HTML>\n<BODY>\n <tr valign=""top""> \n <td ...",1


In [9]:
import string
from nltk.corpus import stopwords

def processText(text):
    noPunctuation = [char for char in text if char not in string.punctuation]
    noPunctuation = ''.join(noPunctuation)
    
    return ' '.join([word for word in noPunctuation.split() if word.lower() not in stopwords.words('english')])

all_data['body']=all_data['body'].apply(processText)
all_data['body'].tail()

353    stumbling greatest way marketing century undou...
354    stumbling greatest way marketing century undou...
355    html head meta httpequivContentLanguage conten...
356    HTML BODY tr valigntop td height295 bgcolorFFF...
357    UNCOMMON EXOTIC PLEASURE BOTANICALS Feeling ma...
Name: body, dtype: object

In [10]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stemming (text):
    return ''.join([stemmer.stem(word) for word in text])

all_data['body']=all_data['body'].apply(stemming)
all_data['body'].tail()

353    stumbling greatest way marketing century undou...
354    stumbling greatest way marketing century undou...
355    html head meta httpequivcontentlanguage conten...
356    html body tr valigntop td height295 bgcolorfff...
357    uncommon exotic pleasure botanicals feeling ma...
Name: body, dtype: object

In [11]:
vectorizer = CountVectorizer(stop_words='english')
X_few = hamEmails[1]
count_matrix = vectorizer.fit_transform(X_few)
count_array = count_matrix.toarray()
print(count_array)
df4 = pd.DataFrame(data = count_array,columns = vectorizer.get_feature_names_out())
df4

all_data=df.append(dfSpam)
all_data

[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

,To,From,Subject,Content-Type,body,isSpam
0,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,Robert Elz <kre@munnari.OZ.AU>,Re: New Sequences Window,"text/plain; charset=""us-ascii""","Date: Wed, 21 Aug 2002 10:54:46 -05...",0
3,undisclosed-recipient:;,Monty Solomon <monty@roscom.com>,[IRR] Klez: The Virus That Won't Die,"text/plain; charset=""us-ascii""",Klez: The Virus That Won't Die\n \nAlready the...,0
11,Chris Garrigues <cwg-dated-1030376441.95dd45@D...,Robert Elz <kre@munnari.OZ.AU>,Re: New Sequences Window,"text/plain; charset=""us-ascii""","Date: Wed, 21 Aug 2002 10:40:39 -05...",0
14,linux-raid@vger.kernel.org,"""John P. Looney"" <valen@tuatha.org>",[ILUG] Re: Problems with RAID1 on cobalt raq3,"text/plain; charset=""us-ascii""","On Mon, Aug 19, 2002 at 03:08:16PM +0100, John...",0
18,"Kiall Mac Innes <kiall@redpie.com>, ILUG <ilug...",Fergal Moran <fergal.moran@wasptech.com>,RE: [ILUG] Sun Solaris..,"text/plain; charset=""us-ascii""",In a nutshell - Solaris is Suns own flavour of...,0
...,...,...,...,...,...,...
353,"ejennifer3@uswest.net, as99@hotmail.com, nvg_c...",2002biz2biz2513@Flashmail.com,FWD:Direct marketing is working ...,"text/plain; charset=""Windows-1252""",There is NO stumbling on to it! \n\nThe greate...,1
354,"ejennifer3@uswest.net, as99@hotmail.com, nvg_c...",biz2biz2446@Flashmail.com,See your Company sales sky rocket. ...,"text/plain; charset=""Windows-1252""",There is NO stumbling on to it! \n\nThe greate...,1
355,undisclosed-recipients:;,Answer.Us@davicom.co.kr,$10 a hour for watching e-mmercials! No joke!,text/html,"<html>\n\n<head>\n<meta http-equiv=""Content-La...",1
356,Mike@dogma.slashnull.org,beautyinfufuxxxmeb13mxy@aol.com,Make a Fortune On eBay ...,"text/html; charset=""iso-8859-1""","<HTML>\n<BODY>\n <tr valign=""top""> \n <td ...",1


In [12]:
print("Using Count Vectorizer")
X = vectorizer.fit_transform(all_data["body"])
y = all_data["isSpam"]

X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y, test_size=0.3, random_state=42)

clf = GaussianNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
print("Accuracy score for Gaussian Naive Baye's is : ", clf.score(X_test, y_test))


train_x,test_x,train_y,test_y = train_test_split(X,y,test_size = 0.25)
from sklearn.metrics import accuracy_score

mnb = MultinomialNB(alpha=1.9)        
mnb.fit(train_x,train_y)
y_pred1 = mnb.predict(test_x)
print("Accuracy Score for Multinomial Naive Baye's is : ", accuracy_score(y_pred1,test_y))

rfc = RandomForestClassifier(n_estimators=100,criterion='gini')
rfc.fit(train_x,train_y)
y_pred3 = rfc.predict(test_x)
print("Accuracy Score for Random Forest Classifier is : ", accuracy_score(y_pred3,test_y))


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(all_data.body,all_data.isSpam,test_size=0.25)


X = vectorizer.fit_transform(all_data["body"])
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), all_data.isSpam, test_size=0.3, random_state=0)

from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',GaussianNB())
])


from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, X_train, y_train, cv=3)
score.mean()
print("Accuracy score for the mean of all folds using cross validation for Logistic Regression is : ", score.mean())

Using Count Vectorizer
Accuracy score for Gaussian Naive Baye's is :  0.957983193277311
Accuracy Score for Multinomial Naive Baye's is :  0.9797979797979798
Accuracy Score for Random Forest Classifier is :  0.9747474747474747
Accuracy score for the mean of all folds using cross validation for Logistic Regression is :  0.9674794359576969


In [13]:
print("\nUsing TFIDF Vectorizer")
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf.fit(all_data['body'])
#print(tfidf.idf_)

vector = tfidf.transform(all_data['body'])
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), all_data.isSpam, test_size=0.3, random_state=0)
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('nb',GaussianNB())
])

clf = GaussianNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
print("Accuracy Score for Gaussian Naive Baye's is : ",clf.score(X_test,y_test))

from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('nb',MultinomialNB())
])

mnb = MultinomialNB(alpha=1.9)        
mnb.fit(X_train,y_train)
y_pred1 = mnb.predict(X_test)
print("Accuracy Score for Multinomial Naive Baye's : ", accuracy_score(y_pred1,y_test))


Using TFIDF Vectorizer
Accuracy Score for Gaussian Naive Baye's is :  0.9705882352941176
Accuracy Score for Multinomial Naive Baye's :  0.9537815126050421
